In [3]:
from pyspark.sql import DataFrame, SparkSession

spark: SparkSession = (
    SparkSession.builder.master("local[3]")  # type: ignore
    .appName("Aggregate Example")
    .getOrCreate()
)

spark.version

25/04/08 11:46:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.5.5'

In [6]:
invoice_df: DataFrame = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/invoices.csv")
)

# Simple Aggregations

summarize whole dataframe to get 1 single row in the result.

> these aggregates are functions, so we can use them in column object expression or SQL like string expression.


In [13]:
from pyspark.sql import functions as F

invoice_df.select(
    count("*").alias("Total Count"),
    F.sum("Quantity").alias("Total Quantity"),
    F.avg("UnitPrice").alias("Average Unit Price"),
    F.count_distinct("InvoiceNo").alias("Distinct Invoice Count"),
).show()

[Stage 10:===================>                                      (1 + 2) / 3]

+-----------+--------------+------------------+----------------------+
|Total Count|Total Quantity|Average Unit Price|Distinct Invoice Count|
+-----------+--------------+------------------+----------------------+
|     541909|       5176450| 4.611113626088481|                 25900|
+-----------+--------------+------------------+----------------------+



In [14]:
invoice_df.selectExpr(
    "COUNT(1) AS `Total Count`",  # count even null values, COUNT(*)
    "COUNT(StockCode) AS `Total Stock Count`",  # count non-null values
    "SUM(Quantity) AS `Total Quantity`",
    "AVG(UnitPrice) AS `Average Unit Price`",
).show()

[Stage 16:>                                                         (0 + 3) / 3]

+-----------+-----------------+--------------+------------------+
|Total Count|Total Stock Count|Total Quantity|Average Unit Price|
+-----------+-----------------+--------------+------------------+
|     541909|           541908|       5176450| 4.611113626086849|
+-----------+-----------------+--------------+------------------+



## Exercise

- based on this [_dataset_](code/01-Aggregate/data/invoices.csv)

<p align="center">
    <img src="https://github.com/user-attachments/assets/7db0d2f0-dc88-4f3b-8e88-52a1fa7b9f21" width="50%">
</p>

- group by `Country` and `InvoiceNo`
- `InvoiceValue` is sum of [`Quantity` * `UnitPrice`]